In [2]:
import urllib.request 
import re
from elasticsearch import Elasticsearch

In [3]:
import requests
# run bin/elasticsearch in elasticsearch directory in terminal before running this cell
res = requests.get('http://localhost:9200')

In [4]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

In [5]:
def test_ES(es):
    return es.ping()  # got True

In [6]:
if test_ES(es):
    print('ES instance working')
else:
    print('ES instance not working')

ES instance working


In [7]:
def index_info(index_name):
    count, deleted, shards, =  es.cat.indices(index=index_name, h=['docs.count', 'docs.deleted', 'pri'])[:-1].split(' ')
    print(
      """
      #### INDEX INFO #####
      index_name = {}
      doc_count = {}
      shard_count = {}
      deleted_doc_count = {}
      """.format(index_name, count, shards, deleted)
  )

In [8]:
import pandas as pd
# read data from steam data directory 
games = pd.read_pickle('final_data.pickle')

In [29]:
games

,type,name,steam_appid,developers,publishers,categories,genres,about_the_game_clean,short_description_clean,release_date,about_the_game
0,game,Counter-Strike,10,[Valve],[Valve],"[Multi-player, PvP, Online PvP, Shared/Split S...",[Action],play world number 1 online action game engage ...,play world number 1 online action game engage ...,2000-11-01,Play the world's number 1 online action game. ...
1,game,Team Fortress Classic,20,[Valve],[Valve],"[Multi-player, PvP, Online PvP, Shared/Split S...",[Action],one popular online action game time team fortr...,one popular online action game time team fortr...,1999-04-01,One of the most popular online action games of...
2,game,Day of Defeat,30,[Valve],[Valve],"[Multi-player, Valve Anti-Cheat enabled]",[Action],enlist intense brand axis v allied teamplay se...,enlist intense brand axis v allied teamplay se...,2003-05-01,Enlist in an intense brand of Axis vs. Allied ...
3,game,Deathmatch Classic,40,[Valve],[Valve],"[Multi-player, PvP, Online PvP, Shared/Split S...",[Action],enjoy fast paced multiplayer gaming deathmatch...,enjoy fast paced multiplayer gaming deathmatch...,2001-06-01,Enjoy fast paced multiplayer gaming with Death...
4,game,Half-Life: Opposing Force,50,[Gearbox Software],[Valve],"[Single-player, Multi-player, Valve Anti-Cheat...",[Action],return black mesa research facility one milita...,return black mesa research facility one milita...,1999-11-01,Return to the Black Mesa Research Facility as ...
...,...,...,...,...,...,...,...,...,...,...,...
57307,game,Football tournament,1691700,[Phoenixxx Games],[Phoenixxx Games],[Single-player],"[Casual, Indie, Simulation, Sports]",football united country world football field n...,football united country world football field n...,2021-07-02,Football has united all the countries of the w...
57308,game,Survival Escape Room,1691760,[Braden Frye],[Frye Games],[Single-player],[Indie],welcome survival escape room youve kidnapped m...,youve kidnapped much time escape keep mind one...,2021-07-23,"Welcome to Survival Escape Room, You've been k..."
57311,game,Cavern Commandos,1692480,[Soiree Games],[Soiree Games],"[Single-player, Full controller support]","[Action, Indie]",inspired classic platformers like rick dangero...,inspired classic platformers like rick dangero...,2021-07-06,Inspired by classic platformers like Rick Dang...
57312,game,Hidden Shapes Black Skull - Jigsaw Puzzle Game,1692580,[YAW Studios],[YAW Studios],"[Single-player, Steam Achievements]","[Casual, Indie]",disconnect world game beautiful art inspired w...,relax beautiful hand drawn puzzle inspired wor...,2021-08-06,Disconnect yourself from the world with this g...


In [30]:
games.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49317 entries, 0 to 57313
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   type                     49317 non-null  object        
 1   name                     49317 non-null  object        
 2   steam_appid              49317 non-null  int64         
 3   developers               49317 non-null  object        
 4   publishers               49317 non-null  object        
 5   categories               49317 non-null  object        
 6   genres                   49317 non-null  object        
 7   about_the_game_clean     49317 non-null  object        
 8   short_description_clean  49317 non-null  object        
 9   release_date             49317 non-null  datetime64[ns]
 10  about_the_game           49317 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(9)
memory usage: 5.5+ MB


In [18]:
# es.indices.delete(index='steam_test')

{'acknowledged': True}

In [19]:
request_body = {
    'settings': {
        'number_of_shards': 1,
        'number_of_replicas': 1,
        
    },
    'mappings': {
          'properties': {
              'title': {'type': 'text'},
              'App_ID':{'type': 'long'},
              'developer': {'type': 'text'},
              'publisher': {'type': 'text'},
              'categories': {'type': 'keyword'},
              'genres': {'type': 'keyword'},
              'body': {'type': 'text'},
              'abstract': {'type': 'text'},
              'release_date': {
                  'type': 'date',
                  'format':'yyyy-mm-dd'},
          }
    }
}

index_name = 'steam_test'
try:
    es.indices.get(index_name)
    print('index {} already exists'.format(index_name))
except:
    print('creating index {}'.format(index_name))
    es.indices.create(index_name, body=request_body)
    print('{} index has been created'.format(index_name))

creating index steam_test
steam_test index has been created


In [26]:
print('legnth: ',len(games.values[0,1:-1]))
games.values[0,1:-1]

legnth:  9


array(['Counter-Strike', 10, list(['Valve']), list(['Valve']),
       list(['Multi-player', 'PvP', 'Online PvP', 'Shared/Split Screen PvP', 'Valve Anti-Cheat enabled']),
       list(['Action']),
       'play world number 1 online action game engage incredibly realistic brand terrorist warfare wildly popular team based game ally teammate complete strategic mission take enemy site rescue hostage role affect team success team success affect role',
       'play world number 1 online action game engage incredibly realistic brand terrorist warfare wildly popular team based game ally teammate complete strategic mission take enemy site rescue hostage role affect team success team success affect role',
       Timestamp('2000-11-01 00:00:00')], dtype=object)

In [27]:
for i, (title, App_ID, developer, publisher, categories, genres, body, abstract, release_date) in enumerate(games.values[:,1:-1]):
    doc_body = {
            'title': title,
            'App_ID': App_ID,
            'developer': developer,
            'publisher': publisher,
            'categories': categories,
            'genres': genres,
            'body': body,
            'abstract': abstract,
            'release_date': release_date
              }
    es.index(index_name, doc_body, id=i)

RequestError: RequestError(400, 'mapper_parsing_exception', "failed to parse field [release_date] of type [date] in document with id '0'. Preview of field's value: '2000-11-01T00:00:00'")

In [109]:
print('we have made and index called {} with {} documents'.format(index_name, es.cat.count(index=index_name,h=['count'])))
index_info(index_name)

we have made and index called steam_test with 56122
 documents

      #### INDEX INFO #####
      index_name = steam_test
      doc_count = 56122
      shard_count = 1
      deleted_doc_count = 0
      


In [11]:
es.get(index=index_name, id=50)

{'_index': 'steam_test',
 '_type': '_doc',
 '_id': '50',
 '_version': 1,
 '_seq_no': 50,
 '_primary_term': 1,
 'found': True,
 '_source': {'App_ID': 2290,
  'title': 'Final DOOM',
  'abstract': 'Two New, 32-Level DOOM II Episodes. Evilution: Far from earth, the UAC recommenced their experiments on on of the moons of Jupiter. A spaceship, mistaken for a supply vexxel on radar, hovered above the base. Hideous demons poured out, blanketing the base with death. All your comrades were quickly slaughtered or zombified.',
  'body': "Two New, 32-Level DOOM II Episodes.Evilution:Far from earth, the UAC recommenced their experiments on on of the moons of Jupiter. A spaceship, mistaken for a supply vexxel on radar, hovered above the base. Hideous demons poured out, blanketing the base with death. All your comrades were quickly slaughtered or zombified. This time, it's not about survival. It's about revenge.The Plutonia Experiment:Every effort has been made by the nation's top scientists to close 

In [95]:
query_body = {
    'query':{
        'term': {
            'body':  'ship'
        }
    }
}
print('### RESULTS ####')
explain=True
results = es.search(index=index_name, body=query_body, explain=explain)['hits']['hits']
for hit in results:
    print('title: {} - score: {}'.format(hit['_source']['title'], hit['_score']))
if explain:
    print('some info on results')
    print(hit['_explanation'])

### RESULTS ####
title: Poly Pirates - score: 6.499002
title: Bob Came in Pieces - score: 6.425233
title: Deep Space Battle Simulator - score: 6.351606
title: V ARRR - score: 6.2367477
title: 1912 Titanic Mystery - score: 6.2364287
title: SEX Cruise VR - score: 6.2364287
title: Rocket Squad - score: 6.191264
title: Captivus - score: 6.17612
title: Adrorium - score: 6.1709504
title: PXL-SHIP - score: 6.168356
some info on results
{'value': 6.168356, 'description': 'weight(body:ship in 41823) [PerFieldSimilarity], result of:', 'details': [{'value': 6.168356, 'description': 'score(freq=4.0), computed as boost * idf * tf from:', 'details': [{'value': 2.2, 'description': 'boost', 'details': []}, {'value': 3.2122538, 'description': 'idf, computed as log(1 + (N - n + 0.5) / (n + 0.5)) from:', 'details': [{'value': 2257, 'description': 'n, number of documents containing term', 'details': []}, {'value': 56064, 'description': 'N, total number of documents with field', 'details': []}]}, {'value':